<a href="https://colab.research.google.com/github/mz-zarei/DL1_COVID_XRAY/blob/main/COVID_XRAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1-Importing Libraries and Unzipping Data** adjust the address to the submitted zip file (DL1_MZarei.rar) and zipped data (DataChallenge1.rar).

In [ ]:
import os
os.mkdir("MZareiFolder") 
os.mkdir("MZareiFolder/MyData") 
os.mkdir("MZareiFolder/MyData/Class1") 
os.mkdir("MZareiFolder/MyData/Class0") 
os.makedirs('MZareiFolder/MyData/splited')
os.makedirs('MZareiFolder/MyData/splited/train')
os.makedirs('MZareiFolder/MyData/splited/val')

# Unzipping the submitted RAR file including data and models
!unrar x "/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/DL1_MZarei.rar" "MZareiFolder"

# Adjust the path to the zipped data file
!unrar x "/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/DataChallenge1.rar" "MZareiFolder"

# Adjust the path where the model should be saved
save_model_path = "MZareiFolder/"





Streaming output truncated to the last 5000 lines.
Extracting  MZareiFolder/train/train/54024041-60394.jpg                   66%  OK 
Extracting  MZareiFolder/train/train/540258098-21839.jpg                  66%  OK 
Extracting  MZareiFolder/train/train/540350483-41806.jpg                  66%  OK 
Extracting  MZareiFolder/train/train/540431093-13429.jpg                  66%  OK 
Extracting  MZareiFolder/train/train/540536998-32076.jpg                  66%  OK 
Extracting  MZareiFolder/train/train/54067835-4714.jpg                    66%  OK 
Extracting  MZareiFolder/train/train/540726111-30138.jpg                  66%  OK 
Extracting  MZareiFolder/train/train/540845213-49229.jpg                  66%  OK 
Extracting  MZareiFolder/train/train/540921514-12237.jpg                  66%  OK 
Extracting  MZareiFolder/train/train/541028079-62927.jpg                  66%  OK 
Extracting  MZareiFolder/trai

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from tqdm import tqdm
from torch.utils.data import Dataset, TensorDataset
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
import pandas as pd
import argparse
import cv2
import os, shutil
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import *
from collections import OrderedDict
import torchvision.models as models
from torch.autograd import Variable
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score 
import random
from collections import Counter 



## Tried to Fix all seeds but unsuccessful!

seed = 1
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.enabled = False

In [ ]:
!pip install split_folders
import splitfolders

## **2-Data Preperation** Including Validation/Train split and creating DataLoaders

In [ ]:
## Arrangeing DataSet so that pytorch ImageFolder can be used

imagePaths = list(paths.list_images("MZareiFolder/train/train"))
train_labels = pd.read_csv("MZareiFolder/train_labels.csv")



for imagePath in tqdm(imagePaths):

    image_name = imagePath.split(os.path.sep)[-1]
    # find image lable in train_labels
    image_label = train_labels.loc[train_labels['File'] == image_name, 'Label'].iloc[0]
    if image_label == 1:
        shutil.copy(imagePath, "MZareiFolder/MyData/Class1" + "/" + image_name)
    if image_label == 0:
        shutil.copy(imagePath, "MZareiFolder/MyData/Class0" + "/" + image_name)

print(len(list(paths.list_images("MZareiFolder/MyData/Class0"))))
print(len(list(paths.list_images("MZareiFolder/MyData/Class1"))))

100%|██████████| 15264/15264 [00:19<00:00, 769.13it/s]

13694
1570


In [ ]:
# Split Data into train/Validation

splitfolders.ratio("MZareiFolder/MyData", output='MZareiFolder/MyData/splited', seed=1, ratio=(0.90, 0.1))

Copying files: 15264 files [00:02, 7027.56 files/s]


In [ ]:
print('train data:')
print(len(list(paths.list_images("MZareiFolder/MyData/splited/train/Class0"))))
print(len(list(paths.list_images("MZareiFolder/MyData/splited/train/Class1"))))

print('Validation data:')
print(len(list(paths.list_images("MZareiFolder/MyData/splited/val/Class0"))))
print(len(list(paths.list_images("MZareiFolder/MyData/splited/val/Class1"))))

train data:
12324
1413
Validation data:
1370
157


In [ ]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation((-5,5)),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

transform_val = transforms.Compose([
    transforms.Grayscale(3),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

TrainData = datasets.ImageFolder(root="MZareiFolder/MyData/splited/train", transform=transform_train)
ValidationData = datasets.ImageFolder(root="MZareiFolder/MyData/splited/val", transform=transform_val)


sampler = torch.utils.data.sampler.WeightedRandomSampler([1,8], num_samples=2, replacement=True)

train_loader=torch.utils.data.DataLoader(TrainData,batch_size=8, shuffle=sampler)
val_loader=torch.utils.data.DataLoader(ValidationData, batch_size=32, shuffle=False)

# print("Val batch number: ", len(list(enumerate(val_loader))))


## **3- Defining Train and Evaluation Functions and Mixup data augmentation**

In [ ]:
def mixup_data(x, y, alpha=0.5, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

criterion = nn.CrossEntropyLoss()

def train(model, dataloader):
    optimizer = optim.Adam(model.parameters(), lr=0.0001, eps = 0.0001/epochs)
    model.train()
    running_loss = 0
    total_steps = 0
    for i, (images, labels) in tqdm(enumerate(dataloader)):
        images = images.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()
        
        inputs, targets_a, targets_b, lam = mixup_data(images, labels, 0.5, True)
        inputs, targets_a, targets_b = map(Variable, (inputs, targets_a, targets_b))



        output = model(inputs)
        # loss = F.cross_entropy(output, labels, weight=torch.FloatTensor([1,2]).to(device))
        # loss = F.cross_entropy(output, labels)
        loss = mixup_criterion(criterion, output, targets_a, targets_b, lam)
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
        total_steps += 1
    
    return running_loss/total_steps
    

def evaluate(model, dataloader):

    model.eval()
    correct = 0
    test_loss = 0
    pred_list = np.array([])
    label_list = np.array([])

    with torch.no_grad():
        for i, (images, labels) in tqdm(enumerate(dataloader)):
            images = images.to(device)
            labels = labels.to(device)
            
            output = model(images)
            test_loss += F.cross_entropy(output, labels, reduction = 'sum').item()

            prd = output.argmax(1)
            label_list= np.append(label_list, labels.cpu().numpy())
            pred_list = np.append(pred_list, prd.cpu().numpy())

            correct += torch.sum(labels == prd)
    
    test_loss /= len(dataloader.dataset)
    accuracy = accuracy_score(label_list, pred_list)
    f1 = f1_score(label_list, pred_list)
    recall = recall_score(label_list, pred_list)
    precision = precision_score(label_list, pred_list)
    return test_loss, accuracy, f1, recall, precision

## **4-MODEL ARCHITECTURE**

In [ ]:
# model = models.googlenet(True)
# for param in model.parameters():
#     param.requires_grad = False
# ## Re-designing the classifier head
# classifier = nn.Sequential(OrderedDict([    
#                         ('fc1', nn.Linear(1024, 256)),
#                         ('relu', nn.ReLU()), 
#                         ('dropout1', nn.Dropout(0.3)),
#                         ('fc2', nn.Linear(256, 128)),
#                         ('dropout2', nn.Dropout(0.3)),
#                         ('relu', nn.ReLU()),
#                         ('fc3', nn.Linear(128, 2)),
#                         ('output', nn.LogSoftmax())
#                         ]))

# model.fc = classifier





## Loading AlexNet weights
model = models.alexnet(True)

for param in model.parameters():
    param.requires_grad = False


## Re-designing the classifier head
classifier = nn.Sequential(OrderedDict([    
                        ('flat', nn.Flatten()),  
                        ('dropout1', nn.Dropout(0.3)),          
                        ('fc1', nn.Linear(9216, 256)),
                        ('relu', nn.ReLU()), 
                        ('dropout1', nn.Dropout(0.3)),
                        ('fc2', nn.Linear(256, 128)),
                        ('dropout2', nn.Dropout(0.3)),
                        ('relu', nn.ReLU()),
                        ('fc3', nn.Linear(128, 2)),
                        ('output', nn.LogSoftmax())
                        ]))

model.classifier = classifier

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(model)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth



AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (flat): Flatten(start_dim=1, end_dim=-1)
    (dropout1): Dropout(p=0.3, inplace=False)
    (fc1): L

## **5-TRAIN MODEL**

In [ ]:
epochs = 30

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Cuda available: ",torch.cuda.is_available())
print("Current device: ",  torch.cuda.current_device())


if device.type == 'cuda':
    torch.cuda.manual_seed_all(seed)


model.to(device)
max_f1 = 0

losses = []
for e in range(epochs):
    print(f"\nepoch {e+1}/{epochs}")
    loss = train(model, train_loader)
    print(f" loss = {loss}")
    losses.append(loss)
    val_loss, val_accuracy, val_f1, val_recall, val_precision = evaluate(model, val_loader)
    # train_loss, train_accuracy, train_f1, _ = evaluate(model, train_loader)

    
    if val_f1 >= max_f1:
        torch.save(model, save_model_path + 'best-model-max-f1.pt') 
        max_f1 = val_f1

    # torch.save(model, save_model_path + 'model-e'+ str(e+1)+ '.pt')

    print(" ")
    print("Validation loss: ", val_loss, "/ Accuracy: ", val_accuracy, " / f1: ", val_f1, " / recall: ", val_recall, " / precision: ", val_precision)
    if e+1 == 6:
        for param in model.parameters():
            param.requires_grad = True







Cuda available:  True
Current device:  0


0it [00:00, ?it/s]


epoch 1/30


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
1718it [02:11, 13.03it/s]
0it [00:00, ?it/s]

 loss = 0.19924356162247744


...

 
Validation loss:  0.03571736511055285 / Accuracy:  0.9862475442043221  / f1:  0.9345794392523364  / recall:  0.9554140127388535  / precision:  0.9146341463414634

epoch 30/30


1718it [02:43, 10.54it/s]
0it [00:00, ?it/s]

 loss = 0.09192268888286483


48it [00:13,  3.48it/s]

 
Validation loss:  0.038976718557567944 / Accuracy:  0.9862475442043221  / f1:  0.9329073482428115  / recall:  0.9299363057324841  / precision:  0.9358974358974359


## **6-Prepare and Predict Test Data**

In [ ]:
model = torch.load("/content/MZareiFolder/DL1_MZarei/Models/model6.pt")  
model.eval()


loader = transforms.Compose([
    transforms.Grayscale(3),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = cv2.imread(image_name)
    image = Image.fromarray(image)

    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  
    return image.cuda()  




test_image_names = []
test_labels_pred = []
pred_prob = []
test_Paths = list(paths.list_images("MZareiFolder/test/test"))


for test_Path in tqdm(test_Paths):
    image_name = test_Path.split(os.path.sep)[-1]

    # load image and change color channels
    image = image_loader(test_Path) 

    output = model(image)
    prd = output.argmax(1)

    test_image_names.append(image_name)
    test_labels_pred.append(prd.item())


test_image_names = np.array(test_image_names)
test_labels_pred = np.array(test_labels_pred)


output = pd.DataFrame()
output['File'] = test_image_names
output['Label'] = test_labels_pred
output.to_csv('test_pred_labels.csv', index=None)

  0%|          | 0/400 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 400/400 [00:03<00:00, 115.46it/s]


## **Bagging for Validation Data**

In [ ]:

PATH_TO_MODELS = '/content/MZareiFolder/DL1_MZarei/Models'
val_loader=torch.utils.data.DataLoader(ValidationData, batch_size=1, shuffle=False)





def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 



def evaluate_bagging(models_paths, dataloader):

    correct = 0
    test_loss = 0
    pred_list = np.array([])
    label_list = np.array([])
    with torch.no_grad():
        for i, (images, labels) in tqdm(enumerate(dataloader)):
            images = images.to(device)
            labels = labels.to(device)
            prd_all_model = []
            for model_path in models_paths:
                model = torch.load(model_path)
                model.eval()
                output = model(images)
                output = output.argmax(1)
                prd_all_model.append(output.cpu().numpy()[0])

            
            prd = most_frequent(prd_all_model)

            label_list= np.append(label_list, labels.cpu().numpy())
            pred_list = np.append(pred_list, prd)

            correct += torch.sum(labels == prd)
    
    accuracy = accuracy_score(label_list, pred_list)
    f1 = f1_score(label_list, pred_list)
    recall = recall_score(label_list, pred_list)
    precision = precision_score(label_list, pred_list)
    return accuracy, f1, recall, precision, label_list, pred_list





## Getting path to combo models
models_paths_ = list(paths.list_files(PATH_TO_MODELS))
combo = ['model'+item+'.pt' for item in input("Enter the list model e.g.  1 2 3  : ").split()]
print(combo)
models_paths = []
for path in models_paths_:
    model_name = path.split(os.path.sep)[-1]
    if model_name in combo:
        models_paths.append(path)





accuracy, f1, recall, precision, label_list, pred_list = evaluate_bagging(models_paths, val_loader)
print(accuracy, f1, recall, precision)


Enter the list model e.g.  A B C  : K I J H G F L M N O


0it [00:00, ?it/s]

['modelK.pt', 'modelI.pt', 'modelJ.pt', 'modelH.pt', 'modelG.pt', 'modelF.pt', 'modelL.pt', 'modelM.pt', 'modelN.pt', 'modelO.pt']


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
1527it [08:16,  3.07it/s]

0.9927963326784545 0.964401294498382 0.9490445859872612 0.9802631578947368


## **Bagging fot Test Data**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PATH_TO_MODELS = '/content/MZareiFolder/DL1_MZarei/Models'

## Getting paths to combo models
models_paths_ = list(paths.list_files(PATH_TO_MODELS))
combo = ['model'+item+'.pt' for item in input("Enter the list model e.g. 1 2 3 : ").split()]
models_paths = []
for path in models_paths_:
    model_name = path.split(os.path.sep)[-1]
    if model_name in combo:
        models_paths.append(path)


test_Paths = list(paths.list_images("MZareiFolder/test/test"))
print(models_paths)


loader = transforms.Compose([
    transforms.Grayscale(3),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])


def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = cv2.imread(image_name)
    image = Image.fromarray(image)

    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  
    return image.cuda()  




test_image_names = []
test_labels_pred = []
pred_prob = []
for test_Path in tqdm(test_Paths):
    image_name = test_Path.split(os.path.sep)[-1]

    # load image and change color channels
    image = image_loader(test_Path)

    
    prd_all_model = []
    for model_path in models_paths:
        model = torch.load(model_path)
        model.eval()
        output = model(image)
        output = output.argmax(1)
        prd_all_model.append(output.cpu().numpy()[0])

    prd = most_frequent(prd_all_model)

    test_image_names.append(image_name)
    test_labels_pred.append(prd)


test_image_names = np.array(test_image_names)
test_labels_pred = np.array(test_labels_pred)


output = pd.DataFrame()
output['File'] = test_image_names
output['Label'] = test_labels_pred
output.to_csv('test_pred_labels_ABC.csv', index=None)

Enter the list model e.g. A B C : K I J H G F L M N O


  0%|          | 0/400 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelG.pt', '/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelH.pt', '/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelF.pt', '/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelJ.pt', '/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelI.pt', '/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelK.pt', '/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelL.pt', '/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelM.pt', '/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelN.pt', '/content/drive/MyDrive/Colab Notebooks/COVID_DataChallenge1/Models/modelO.pt']


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 400/400 [02:07<00:00,  3.13it/s]
